In [1]:
push!(LOAD_PATH, normpath(@__DIR__, "../../", "src/models"));
push!(LOAD_PATH, normpath(@__DIR__, "../../", "src/processing"));
ENV["COLUMNS"] = 200;

In [2]:
using Dates
using JuMP
using CSV
using DataFrames
using LinearAlgebra

In [3]:
using BedsData
using ForecastData
using GeographicData

┌ Info: Precompiling BedsData [top-level]
└ @ Base loading.jl:1260


In [4]:
using PatientAllocation

In [5]:
states = ["CT", "DE", "MA", "MD", "ME", "NH", "NJ", "NY", "PA", "RI"]

start_date = Date(2020, 5, 1)
end_date   = Date(2020, 5, 30)

pct_beds_available = 0.25
travel_threshold_hours = 4.0
hospitalized_days = 14;

In [6]:
N = length(states);
T = (end_date - start_date).value + 1;

In [7]:
forecast_admitted = forecast(
    states, start_date, end_date,
    level=:state,
    source=:ihme,
    forecast_type=:admitted,
    patient_type=:regular,
    bound_type=:mean,
);

In [8]:
forecast_initial = forecast(
    states, start_date-Dates.Day(1), start_date-Dates.Day(1),
    level=:state,
    source=:ihme,
    forecast_type=:active,
    patient_type=:regular,
    bound_type=:mean,
)[:];

In [9]:
forecast_discharged = forecast(
    states, start_date-Dates.Day(hospitalized_days), start_date-Dates.Day(1),
    level=:state,
    source=:ihme,
    forecast_type=:admitted,
    patient_type=:regular,
    bound_type=:mean,
)
forecast_discharged = hcat(forecast_discharged, zeros(Float32, N, T - hospitalized_days));

In [10]:
beds = n_beds(states, bed_type=:all, pct_beds_available=pct_beds_available);
adj = adjacencies(states, level=:state, source=:google, threshold=travel_threshold_hours);

In [11]:
model = patient_allocation(
    beds,
    forecast_initial,
    forecast_admitted,
    forecast_discharged,
    adj,
    hospitalized_days=hospitalized_days,
    send_new_only=true,
    sendrecieve_switch_time=3,
    min_send_amt=10,
    smoothness_penalty=0.001,
    setup_cost=0,
    sent_penalty=0,
    verbose=true
)
sent = value.(model[:sent])
println("termination status: ", termination_status(model))
println("solve time: ", round(solve_time(model), digits=3), "s")
println("objective function value: ", round(objective_value(model), digits=3))

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 7604 rows, 7360 columns and 123280 nonzeros
Model fingerprint: 0x9ea9a10a
Model has 580 SOS constraints
Variable types: 4360 continuous, 0 integer (0 binary)
Semi-Variable types: 3000 continuous, 0 integer
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-03, 1e+00]
  Bounds range     [1e+01, 1e+01]
  RHS range        [4e-01, 1e+04]
Presolve removed 3240 rows and 2983 columns
Presolve time: 0.22s
Presolved: 7484 rows, 5937 columns, 71399 nonzeros
Presolved model has 492 SOS constraint(s)
Variable types: 4347 continuous, 1590 integer (1590 binary)
Found heuristic solution: objective 49330.345550

Root relaxation: objective 8.402048e+03, 3067 iterations, 0.23 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    B

In [12]:
overflow_per_day = (i,t) -> sum(max.(0,
    forecast_initial[i] - sum(forecast_discharged[i,1:min(t,hospitalized_days)])
    + sum(forecast_admitted[i,max(1,t-hospitalized_days):t])
    - sum(sent[i,:,1:t-1])
    + sum(sent[:,i,max(1,t-hospitalized_days):t])
    - beds[i])
)
overflow = i -> sum(overflow_per_day(i,t) for t=1:size(sent,3));

In [13]:
println("Total overflow: ", sum(overflow.(1:length(states))))

Total overflow: 8399.8662109375


In [14]:
summary = DataFrame(
    state=states,
    total_sent=sum(sent, dims=[2,3])[:],
    total_received=sum(sent, dims=[1,3])[:],
    overflow=overflow.(1:length(states)),
)

,state,total_sent,total_received,overflow
,String,Float64,Float64,Float64
1,CT,5529.05,0.0,769.123
2,DE,0.0,478.873,0.0
3,MA,0.0,2726.29,0.0
4,MD,4489.73,0.0,181.623
5,ME,0.0,1203.53,0.0
6,NH,0.0,41.6296,0.0
7,NJ,13806.0,0.0,7449.12
8,NY,0.0,17679.9,0.0
9,PA,0.0,1694.51,0.0


In [15]:
sent_matrix = DataFrame(sum(sent, dims=3)[:,:,1])
rename!(sent_matrix, Symbol.(states))
insertcols!(sent_matrix, 1, :state => states)

,state,CT,DE,MA,MD,ME,NH,NJ,NY,PA,RI
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,CT,0.0,0.0,0.0,0.0,1203.53,41.6296,0.0,4283.9,0.0,0.0
2,DE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,MA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,MD,0.0,478.873,0.0,0.0,0.0,0.0,0.0,2316.35,1694.51,0.0
5,ME,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,NH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,NJ,0.0,0.0,2726.29,0.0,0.0,0.0,0.0,11079.7,0.0,0.0
8,NY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
sent_vis_matrix = sum(sent, dims=3)[:,:,1] + diagm(sum(max.(0, forecast_admitted), dims=2)[:] - sum(sent, dims=[2,3])[:])
sent_vis_matrix = DataFrame(sent_vis_matrix)
rename!(sent_vis_matrix, Symbol.(states));

In [17]:
total_patients = (i,t) -> (
    forecast_initial[i] - sum(forecast_discharged[i,1:min(t,hospitalized_days)])
    + sum(forecast_admitted[i,max(1,t-hospitalized_days):t])
    - sum(sent[i,:,1:t])
    + sum(sent[:,i,max(1,t-hospitalized_days):t])
);

In [18]:
outcomes = DataFrame()
for (i,s) in enumerate(states)
    single_state_outcome = DataFrame(
        state=fill(s, T),
        day=start_date .+ Dates.Day.(0:T-1),
        sent=sum(sent[i,:,:], dims=1)[:],
        received=sum(sent[:,i,:], dims=1)[:],
        new_patients=forecast_admitted[i,:],
        total_patients=[total_patients(i,t) for t in 1:T],
        capacity=fill(beds[i], T),
        overflow=[overflow_per_day(i,t) for t in 1:T],
        sent_to=[sum(sent[i,:,t])>0 ? collect(zip(states[sent[i,:,t] .> 0], sent[i,sent[i,:,t].>0,t])) : "[]" for t in 1:T],
        sent_from=[sum(sent[:,i,t])>0 ? collect(zip(states[sent[:,i,t] .> 0], sent[sent[:,i,t].>0,i,t])) : "[]" for t in 1:T],
    )
    outcomes = vcat(outcomes, single_state_outcome)
end
# CSV.write("patient_allocation_results.csv", outcomes)
println("First day:")
filter(row -> row.day == start_date, outcomes)

First day:


,state,day,sent,received,new_patients,total_patients,capacity,overflow,sent_to,sent_from
,String,Date,Float64,Float64,Float32,Float64,Float32,Float64,Any,Any
1,CT,2020-05-01,321.51,0.0,321.51,2132.26,1840.25,613.52,"[(""ME"", 137.084), (""NH"", 41.6296), (""NY"", 142.797)]",[]
2,DE,2020-05-01,0.0,15.9624,34.1128,272.256,506.75,0.0,[],"[(""MD"", 15.9624)]"
3,MA,2020-05-01,0.0,552.85,327.558,2726.09,3997.75,0.0,[],"[(""NJ"", 552.85)]"
4,MD,2020-05-01,216.532,0.0,308.294,2193.84,2228.75,181.623,"[(""DE"", 15.9624), (""NY"", 144.086), (""PA"", 56.4836)]",[]
5,ME,2020-05-01,0.0,137.084,4.4961,170.971,668.0,0.0,[],"[(""CT"", 137.084)]"
6,NH,2020-05-01,0.0,41.6296,32.9998,223.307,584.5,0.0,[],"[(""CT"", 41.6296)]"
7,NJ,2020-05-01,922.172,0.0,922.172,7083.11,4553.25,3452.03,"[(""MA"", 552.85), (""NY"", 369.322)]",[]
8,NY,2020-05-01,0.0,656.205,1051.65,7692.38,10406.2,0.0,[],"[(""CT"", 142.797), (""MD"", 144.086), (""NJ"", 369.322)]"
9,PA,2020-05-01,0.0,56.4836,674.188,4660.07,7980.25,0.0,[],"[(""MD"", 56.4836)]"


In [19]:
s = "NJ"
filter(row -> row.state == s, outcomes)

,state,day,sent,received,new_patients,total_patients,capacity,overflow,sent_to,sent_from
,String,Date,Float64,Float64,Float32,Float64,Float32,Float64,Any,Any
1,NJ,2020-05-01,922.172,0.0,922.172,7083.11,4553.25,3452.03,"[(""MA"", 552.85), (""NY"", 369.322)]",[]
2,NJ,2020-05-02,915.841,0.0,915.841,6000.03,4553.25,2362.62,"[(""MA"", 546.518), (""NY"", 369.322)]",[]
3,NJ,2020-05-03,910.815,0.0,910.815,4961.67,4553.25,1319.23,"[(""MA"", 541.492), (""NY"", 369.322)]",[]
4,NJ,2020-05-04,409.523,0.0,913.432,4458.97,4553.25,315.24,"[(""MA"", 40.2012), (""NY"", 369.322)]",[]
5,NJ,2020-05-05,409.523,0.0,940.136,3982.38,4553.25,0.0,"[(""MA"", 40.2012), (""NY"", 369.322)]",[]
6,NJ,2020-05-06,409.523,0.0,938.541,3475.66,4553.25,0.0,"[(""MA"", 40.2012), (""NY"", 369.322)]",[]
7,NJ,2020-05-07,409.523,0.0,866.922,2835.83,4553.25,0.0,"[(""MA"", 40.2012), (""NY"", 369.322)]",[]
8,NJ,2020-05-08,409.523,0.0,844.234,2119.2,4553.25,0.0,"[(""MA"", 40.2012), (""NY"", 369.322)]",[]
9,NJ,2020-05-09,409.523,0.0,816.379,1355.3,4553.25,0.0,"[(""MA"", 40.2012), (""NY"", 369.322)]",[]


In [20]:
println("Sent to:")
Dict(states[i] => states[row] for (i,row) in enumerate(eachrow(sum(sent, dims=3)[:,:,1] .> 0)))

Sent to:


Dict{String,Array{String,1}} with 10 entries:
  "NH" => String[]
  "CT" => ["ME", "NH", "NY"]
  "RI" => String[]
  "MA" => String[]
  "ME" => String[]
  "NY" => String[]
  "NJ" => ["MA", "NY"]
  "DE" => String[]
  "PA" => String[]
  "MD" => ["DE", "NY", "PA"]